In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/NAMAKAV

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NAMAKAV


In [3]:
#!/usr/bin/env python
# coding: utf-8


In [4]:
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings 
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout
import cv2
import numpy as np
import csv
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)


In [5]:
data = pd.read_csv("data.csv",skiprows = 0, nrows=38023)

result = pd.read_csv("label (Autosaved).csv",skiprows = 0, nrows=38023)




In [6]:
X_Tr,X_Te,Y_Tr,Y_Te=train_test_split(data,result,test_size=0.25,random_state=6)

NET1

In [8]:
model = Sequential()
model.add(Dense(20,input_dim = 4, activation = "relu"))
model.add(Dense(10, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))
model.compile(optimizer='SGD',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 20)                100       
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [10]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

history = model.fit(X_Tr, Y_Tr, epochs=10, batch_size= 10, validation_split = 0.2, callbacks = callbacks_list)

Epoch 1/10
2269/2282 [============================>.] - ETA: 0s - loss: 0.3938 - accuracy: 0.8661
Epoch 1: val_loss improved from inf to 0.38556, saving model to Weights-001--0.38556.hdf5
2282/2282 [==============================] - 8s 4ms/step - loss: 0.3939 - accuracy: 0.8660 - val_loss: 0.3856 - val_accuracy: 0.8704
Epoch 2/10
2270/2282 [============================>.] - ETA: 0s - loss: 0.3938 - accuracy: 0.8661
Epoch 2: val_loss did not improve from 0.38556
2282/2282 [==============================] - 7s 3ms/step - loss: 0.3939 - accuracy: 0.8660 - val_loss: 0.3856 - val_accuracy: 0.8704
Epoch 3/10
2275/2282 [============================>.] - ETA: 0s - loss: 0.3937 - accuracy: 0.8662
Epoch 3: val_loss did not improve from 0.38556
2282/2282 [==============================] - 7s 3ms/step - loss: 0.3939 - accuracy: 0.8660 - val_loss: 0.3856 - val_accuracy: 0.8704
Epoch 4/10
2278/2282 [============================>.] - ETA: 0s - loss: 0.3940 - accuracy: 0.8660
Epoch 4: val_loss did not

In [11]:
scores = model.evaluate(X_Te,Y_Te, verbose=0)
print("loss= ",scores[0],"   ","accuracy= ",scores[1])

loss=  0.39199918508529663     accuracy=  0.8670313358306885


In [12]:
def ScoreImage(imgGray):
    score=[]
    imgGray = imgGray.astype("uint8")
    thresh_img = cv2.threshold(imgGray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    numOfWhite=np.count_nonzero(thresh_img == 255)
    score.append(numOfWhite/(thresh_img.shape[0]*thresh_img.shape[1]))
    return np.max(score)

In [24]:
def predictQulity(X,model):
  print(X)
  predictions  = (model.predict(X) > 0.15).astype(int)
  return predictions


In [25]:
image = pd.read_csv("data_test.csv",skiprows = 1)
p=predictQulity(image,model)
print(p)
print(1-np.count_nonzero(p == 1)/p.size)

   249.4175  46.69432718452651  0.17903645833333334  81.11907298177081
0  251.2415          37.762633             0.151259          80.415069
1  251.3095          57.801400             0.356988         112.062740
2  246.1405          54.415276             0.412435         109.987868
3  251.1445          59.696714             0.505968         134.486881
4  252.5635          52.275750             0.730143         180.014333
5  252.7915          65.362840             0.509549         144.917987
6  246.1695          48.822702             0.355577          91.678709
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
1.0
